# 영어 위키피디아 텍스트 전처리
### 텍스트 전처리

In [1]:
import os
import re
import multiprocessing

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
def sentence_tokenizer(sentence):
    
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(sentence)
    
    result = []
     
    for word in word_tokens:
        if word not in stop_words: 
            result.append(word)
            
    return result

In [3]:
# 파싱한 텍스트를 전처리 한 후 step2 폴더에 저장하는 함수
def corpus_preprocessor(data):
    
    xml_tag = re.compile(r'<[^>]*>')
    allowed_char = re.compile(r'[^A-z^0-9^.^,^?^!^ ]')
    
    path, file_name = data
    print("process file: {}\n".format(file_name))
    with open(os.path.join(path, file_name), 'r', encoding='utf-8') as input:
        with open(os.path.join(os.getcwd(), 'enwiki_step2', file_name), 'w', encoding='utf-8') as output: 
            # Line 단위로 Sentence 취급
            for line in input: 
                
                # [^문자] ^ 뒤의 문자를 제외한 모든 문자 매치
                # re.sub() - 정규 표현식과 일치하는 문자열을 다른 문자열로 대체
                sentence = xml_tag.sub(' ', line)
                allowed_sentence = allowed_char.sub(' ', sentence)
                tokenized_sentence = sentence_tokenizer(allowed_sentence.lower())
                
                output.write(' '.join(tokenized_sentence))
                output.write('\n')

In [4]:
if __name__ == '__main__':
    pool = multiprocessing.Pool(processes=4)
    
    data = [] 
    # path: dirs 와 files 가 있는 경로
    # dirs: path 아래의 폴더들
    # files: path 아래의 파일들
    for path, dirs, files in os.walk('enwiki_step1/'):
        for file_name in files: 
            data.append( (path, file_name) )
        
    # 리스트의 각 요소에 대해 함수 적용
    pool.map(corpus_preprocessor, data) 
    pool.close()
    pool.join()

process file: wiki_08
process file: wiki_18
process file: wiki_12
process file: wiki_17




process file: wiki_03

process file: wiki_14

process file: wiki_16

process file: wiki_04

process file: wiki_09

process file: wiki_10

process file: wiki_13

process file: wiki_02

process file: wiki_11

process file: wiki_07

process file: wiki_19

process file: wiki_05

process file: wiki_15

process file: wiki_06

process file: wiki_00-checkpoint

process file: wiki_00

process file: wiki_01

process file: wiki_03-checkpoint



### W2V 생성

In [4]:
from gensim.models import Word2Vec 

/home/ukyoung/anaconda3/envs/ukyoung/lib/python3.6/site-packages/azure/storage/blob/_encryption.py:19: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  from cryptography.hazmat.backends import default_backend


In [6]:
# 문장을 공백 단위로 끊어서 토큰화
class Loader(object):
    def __init__(self, source_dir):
        self.source_dir = source_dir
        
    def __iter__(self):
        for path, dirs, files in os.walk(self.source_dir):
            for file in files:
                with open(os.path.join(path, file), 'r', encoding='utf-8') as f:
                    for line in f:
                        yield line.replace('\\n', '').replace('?', '').replace('!', '').replace(',', '').replace('.', '').split(' ')

In [7]:
sentences_vocab = Loader('wiki_step2/') # 단어사전 생성을 위한 데이터
sentences_train = Loader('wiki_step2/') # 학습을 위한 데이터

In [20]:
config = {
    'vector_size': 300, # 300차원 Embedding Vector 생성
    'min_count': 5, # 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
    
    'batch_words': 10000, # 사전을 구축할때 한번에 읽을 단어 수
    'workers': multiprocessing.cpu_count(), # 학습을 위한 프로세스의 수
    
    'sg': 1, # 0이면 CBOW, 1이면 skip-gram을 사용한다 
}

In [21]:
model = Word2Vec(**config) # Word2vec 모델 생성
model.build_vocab(sentences_vocab)
print('model.corpus_count: {}'.format(model.corpus_count)) # corpus 개수 카운트

model.train(sentences_train, total_examples=model.corpus_count, epochs=10) # Word2Vec training 

model.save('enwiki_w2v') # 모델을 파일로 저장

model.corpus_count: 11692878


In [5]:
# 모델 로딩 
load_model = Word2Vec.load('enwiki_w2v')

In [14]:
# word vector 출력
print(len(load_model.wv.vectors))

273940
